In [1]:
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.black_litterman import BlackLittermanModel
from pypfopt import black_litterman, risk_models

In [ ]:
viewdict = {"Basis": 0.20, "HP": 0.20, "Momentum": 0.20, "Skewness": 0.20, "Basis-Mom": 0.20}
L = 120
style_ret = style_rets[(style_rets.index>= pd.to_datetime(start)-DateOffset(months=L-1))&(style_rets.index<=end)]
cov = style_ret.fillna(0).rolling(L).cov()
keys = list(sorted(Thetas.keys()))
mu = style_ret.rolling(L).mean().fillna(0)
BOI_weights = dict()
BOI_weights_style = dict()
for key in tqdm(keys):
    if recursive:
        tmp = style_rets.loc[:key].fillna(0)
    else:
        tmp = style_rets[(style_rets.index>= pd.to_datetime(key)-DateOffset(months=L-1))&(style_rets.index<=key)]
    mu = expected_returns.mean_historical_return(tmp.fillna(0),returns_data=True,log_returns=True, frequency=12)
    #S = risk_models.sample_cov(tmp.fillna(0),returns_data=True,log_returns=True, frequency=12)
    S = risk_models.CovarianceShrinkage(tmp.fillna(0),returns_data=True,log_returns=True, frequency=12).ledoit_wolf('single_factor')
    Theta = Thetas[key].fillna(0)

    P = np.array([
        [1,0,0,0,0],
        [0,1,0,0,0],
        [0,0,1,0,0],
        [0,0,0,1,0],
        [0,0,0,0,1]])
    Q = np.array([[0.002],[0.002],[0.002],[0.002],[0.002]]) # 2-vector
    delta = 5
    Omega = BlackLittermanModel.default_omega(S, P = P, tau = 0.3) # 0.01
    bl = BlackLittermanModel(S, P = P, Q = Q, omega = Omega)
    w = np.reshape(np.array(list(bl.bl_weights(delta).values())),(5,1))

#     bl = BlackLittermanModel(S, absolute_views=viewdict)
#     ef = EfficientFrontier(bl.bl_returns(), tmp.fillna(0).cov())
#     raw_weights = ef.max_sharpe()
#     cleaned_weights = ef.clean_weights()
#     w = np.reshape(np.array(list(raw_weights.values())),(5,1))
    BOI_weights[key] = np.reshape((Theta.values @ w), n)
    BOI_weights_style[key] = np.reshape(w,5)
BOI_weights = pd.DataFrame(BOI_weights.values(), index = ret.index[(ret.index>=start)&(ret.index<=end)])
BOI_weights = BOI_weights.fillna(0).apply(lambda x: x.divide(BOI_weights.abs().sum(axis=1,skipna=True)))
BOI_weights.columns = ret.columns
BOI_return = (BOI_weights.shift(1) * ret[(ret.index>=start)&(ret.index<=end)]).fillna(0).sum(axis=1)

BOI_weights_style = pd.DataFrame(BOI_weights_style.values(), index = ret.index[(ret.index>=start)&(ret.index<=end)])
#BOI_weights_style = BOI_weights_style.apply(lambda x: x/sum(abs(x)))
BOI_weights_style = BOI_weights_style.fillna(0).apply(lambda x: x.divide(BOI_weights_style.abs().sum(axis=1,skipna=True)))
BOI_weights_style.columns = style_ret.columns
BOI_style_return = (BOI_weights_style.shift(1) * style_ret[(style_ret.index>=start)&(style_ret.index<=end)]).fillna(0).sum(axis=1)